In [9]:
import os 
import sys 
spark_path = "C:/Users/img/Documents/spark-2.0.2-bin-hadoop2.7"
hadoop_path = "C:\winutils"
os.environ['SPARK_HOME'] = spark_path
os.environ['HADOOP_HOME'] = hadoop_path

sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.9-src.zip")

import pyspark

sc = pyspark.SparkContext('local[*]')

# do something to prove it works
rdd = sc.parallelize(range(1000))
rdd.takeSample(False, 5)

ImportError: No module named 'py4j'